In [113]:
import duckdb as ddb
import pandas as pd
import plotly.graph_objects as go

con = ddb.connect(database=":memory:")

con.sql("""
CREATE TABLE program_beslut AS
    FROM 'data/resultat-ansokningsomgang-2020-2024-beslut.csv';
CREATE TABLE program_kommun AS
    FROM 'data/resultat-ansokningsomgang-2020-2024-diarie_kommun.csv';

CREATE TABLE kurser_beslut AS
    FROM 'data/resultat-for-kurser-inom-yh-2024-beslut.csv';
CREATE TABLE kurser_kommun AS
    FROM 'data/resultat-for-kurser-inom-yh-2024-diarie_kommun.csv';

CREATE TABLE kommun_lan AS
    FROM 'data/kommunlankod-2025.csv';       
""")

In [114]:
def create_df_utbildningsomrade_beslut_single(
    con, beslut=True, utbildningsanordnare: list[str] | None = None, year=2024
):
    rel = con.sql(
        """
    select
        Utbildningsområde,
        count(*) as Beslut
    from kurser_beslut
    where
        ( $utb IS NULL OR Utbildningsanordnare = ANY($utb) )
        and Ansökningsomgång = $year
        and Beslut = $beslut
    group by Utbildningsområde
    order by Beslut desc, Utbildningsområde
    ;""",
        params={"beslut": beslut, "year": year, "utb": utbildningsanordnare},
    )

    df_rel: pd.Dataframe = rel.df().set_index("Utbildningsområde")
    return df_rel.rename(columns={"Beslut": "Beviljade" if beslut else "Avslag"})

In [115]:
def create_df_utbildningsomrade_beslut_merge(con):
    df_utbildningsomrade_beviljade = create_df_utbildningsomrade_beslut_single(con, beslut=True)
    df_utbildningsomrade_avslag = create_df_utbildningsomrade_beslut_single(con, beslut=False)

    df_utbildningsomrade_beslut = pd.merge(
        df_utbildningsomrade_beviljade,
        df_utbildningsomrade_avslag,
        left_index=True,
        right_index=True,
    )

    df_utbildningsomrade_beslut["diff_beslut"] = abs(
        df_utbildningsomrade_beslut["Beviljade"] - df_utbildningsomrade_beslut["Avslag"]
    )

    return df_utbildningsomrade_beslut

In [116]:
def make_fig_utbildningsomrade_beslut(df_utbildningsomrade_beslut):
    fig = go.Figure()

    fig.add_trace(
        go.Bar(
            y=df_utbildningsomrade_beslut.index,
            x=df_utbildningsomrade_beslut["Beviljade"],
            name="Beviljade",
            hovertemplate="%{x}<extra>Beviljade</extra>",
            orientation="h",
            text=df_utbildningsomrade_beslut["Beviljade"],
            textposition="inside",
            insidetextanchor="end",
            textangle=0,
            marker=dict(color="rgba(246, 78, 139, 0.6)"),
        )
    )

    fig.add_trace(
        go.Bar(
            y=df_utbildningsomrade_beslut.index,
            x=df_utbildningsomrade_beslut["diff_beslut"],
            name="Avslag",
            customdata=df_utbildningsomrade_beslut["Avslag"],
            hovertemplate="%{customdata}<extra>Avslag</extra>",
            orientation="h",
            text=df_utbildningsomrade_beslut["Avslag"],
            textposition="inside",
            insidetextanchor="end",
            textangle=0,
            marker=dict(color="rgba(58, 71, 80, 0.6)"),
        )
    )

    sort_order = df_utbildningsomrade_beslut.sort_values("Beviljade").index

    fig.update_layout(
        barmode="stack",
        # yaxis={"categoryorder": "total ascending"},
        yaxis={"categoryorder": "array", "categoryarray": sort_order},
        legend=dict(traceorder="normal"),
    )

    return fig


df_utbildningsomrade_beslut = create_df_utbildningsomrade_beslut_merge(con)
fig_utbildningsomrade_beslut = make_fig_utbildningsomrade_beslut(df_utbildningsomrade_beslut)

fig_utbildningsomrade_beslut.show()

In [117]:
def make_fig_utbildningsomrade_beslut(df_utbildningsomrade_beslut):
    fig = go.Figure()

    fig.add_trace(
        go.Bar(
            y=df_utbildningsomrade_beslut.index,
            x=df_utbildningsomrade_beslut["Avslag"],
            name="Avslag",
            customdata=df_utbildningsomrade_beslut["Avslag"],
            hovertemplate="%{customdata}<extra>Avslag</extra>",
            orientation="h",
            text=df_utbildningsomrade_beslut["Avslag"],
            textposition="inside",
            insidetextanchor="end",
            textangle=0,
            marker=dict(color="rgba(58, 71, 80, 0.6)"),
        )
    )

    fig.add_trace(
        go.Bar(
            y=df_utbildningsomrade_beslut.index,
            x=df_utbildningsomrade_beslut["Beviljade"],
            name="Beviljade",
            hovertemplate="%{x}<extra>Beviljade</extra>",
            orientation="h",
            text=df_utbildningsomrade_beslut["Beviljade"],
            textposition="inside",
            insidetextanchor="end",
            textangle=0,
            marker=dict(color="rgba(246, 78, 139, 0.6)"),
        )
    )

    sort_order = df_utbildningsomrade_beslut.sort_values("Beviljade").index

    fig.update_layout(
        # barmode="stack",
        yaxis={"categoryorder": "array", "categoryarray": sort_order},
        legend=dict(traceorder="normal"),
    )

    return fig


df_utbildningsomrade_beslut = create_df_utbildningsomrade_beslut_merge(con)
fig_utbildningsomrade_beslut = make_fig_utbildningsomrade_beslut(df_utbildningsomrade_beslut)

fig_utbildningsomrade_beslut.show()

In [118]:
def make_fig_utbildningsomrade_beslut(df_utbildningsomrade_beslut):
    fig = go.Figure()

    fig.add_trace(
        go.Bar(
            y=df_utbildningsomrade_beslut.index,
            x=df_utbildningsomrade_beslut["Avslag"],
            name="Avslag",
            # customdata=df_utbildningsomrade_beslut["Avslag"],
            # hovertemplate="%{customdata}<extra>Avslag</extra>",
            hovertemplate="%{x}<extra>Avslag</extra>",
            orientation="h",
            text=df_utbildningsomrade_beslut["Avslag"],
            textposition="inside",
            insidetextanchor="end",
            textangle=0,
            marker=dict(color="rgba(58, 71, 80, 0.6)"),
        )
    )

    fig.add_trace(
        go.Bar(
            y=df_utbildningsomrade_beslut.index,
            x=df_utbildningsomrade_beslut["Beviljade"] - df_utbildningsomrade_beslut["Avslag"],
            name="Beviljade",
            customdata=df_utbildningsomrade_beslut["Beviljade"],
            hovertemplate="%{customdata}<extra>Beviljade</extra>",
            # hovertemplate="%{x}<extra>Beviljade</extra>",
            orientation="h",
            text=df_utbildningsomrade_beslut["Beviljade"],
            textposition="outside",
            insidetextanchor="end",
            textangle=0,
            marker=dict(color="rgba(246, 78, 139, 0.6)"),
        )
    )

    sort_order = df_utbildningsomrade_beslut.sort_values("Beviljade").index

    fig.update_layout(
        barmode="stack",
        # yaxis={"categoryorder": "total ascending"},
        yaxis={"categoryorder": "array", "categoryarray": sort_order},
        legend=dict(traceorder="normal"),
    )

    return fig


df_utbildningsomrade_beslut = create_df_utbildningsomrade_beslut_merge(con)
fig_utbildningsomrade_beslut = make_fig_utbildningsomrade_beslut(df_utbildningsomrade_beslut)

fig_utbildningsomrade_beslut.show()